In [2]:
import pandas as pd
import numpy as np
import csv
from os.path import join as PJOIN

In [18]:
DATA_DIR = "DATA/GENERATED/TRAIN/"
commentTypeClassification = True
DATA_FILES = ["train_libpng_commentType.csv","train_dealii_commentType.csv","train_server_commentType.csv"]
OUTPUT_FILE = "CONCATED_commentType"

In [19]:
SHAPES = set()
def get_all_training_data():
    all_files = []
    if DATA_FILES[0] == 'all':
        for file in os.listdir(DATA_DIR):
            if file[:2] == 'X_':
                all_files.append(file[2:])
    else:
        all_files = DATA_FILES
    
    all_x = []
    all_y = []
    all_z = []
    
    for file in all_files:
        train_x = pd.read_csv(PJOIN(DATA_DIR,"X_"+file),header=None)
        all_x.append(np.array(train_x))
        train_y = pd.read_csv(PJOIN(DATA_DIR,"Y_"+file),header=None)
        all_y.append(train_y)
        train_z = pd.read_csv(PJOIN(DATA_DIR,"Z_"+file),header=None,delimiter='\t')
        SHAPES.add(train_z.shape)
        all_z.append(train_z)
    
    all_x = np.concatenate(all_x)
    all_y = np.concatenate(all_y)
    all_z = np.concatenate(all_z)
    print(all_x.shape,all_y.shape, all_z.shape)
    all_y = all_y.reshape(all_y.shape[0])    
    return all_x, all_y, all_z

In [20]:
# def normalize_data(x):
#     return (x - np.mean(x,axis=0))/np.std(x,axis=0)

def process_y_commentType(y):
    res = []
    for el in y:
        temp = el.strip('][').split(',')
        temp = [int(a) for a in temp]
        res.append(temp)
    return np.array(res)

In [25]:
train_x, train_y, train_z = get_all_training_data()
perm = np.random.permutation(len(train_y))
train_x = train_x[perm]
train_y = train_y[perm]
train_z = train_z[perm]
# Create a file for combined z
with open(DATA_DIR+"Z_"+OUTPUT_FILE+".csv", 'w') as f:
    writer = csv.writer(f, delimiter = '\t')
    header = ['F'+str(1+j) for j in range(len(train_z[0]))]
    writer.writerow(header)
    for el in train_z:
        writer.writerow(el)
if commentTypeClassification:
    train_y = process_y_commentType(train_y)
    print(train_y.shape)
    OUTPUT_TYPES = ['ProgramDomain','ProblemDomain','ProjectManagement']
    for i,ot in enumerate(OUTPUT_TYPES):
        with open(DATA_DIR+OUTPUT_FILE+"_"+ot+".csv",'w') as f:
            writer = csv.writer(f)
            header = ['F'+str(1+j) for j in range(12)]
            header.append('Class')
            header.append('Index')
            writer.writerow(header)
            for j,l in enumerate(train_x):
                writer.writerow(np.append(np.append(l,train_y[j,i]),j))
                

(12774, 12) (12774, 1) (12774, 50)
(12774, 3)


In [17]:
SHAPES

{(3153, 50), (3407, 50), (6214, 49)}

In [23]:
l

array([9.85      , 7.25      , 1.85      , 1.49      , 0.41      ,
       0.05      , 3.65      , 0.05      , 0.41000002, 0.05      ,
       0.05      , 0.05      ])

In [ ]:
tra